In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import sys
path = '/content/gdrive/Team Drives/cs273p project'
sys.path.append(path)

In [3]:
import numpy as np
import pandas as pd

import re
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet') 

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
import pickle

def saveobj(save_list, filename):
    with open(path + '/' + filename, 'wb') as f:  # Python 3: open(..., 'wb')
        pickle.dump(save_list, f)
        
# restore object        
def loadobj(filename):
    with open(path + '/' + filename, 'rb') as f:  # Python 3: open(..., 'rb')
        li = pickle.load(f)
    return li

In [0]:
import data_loader

In [0]:
train_file_path = path + '/data/train.csv'
test_file_path = path + '/data/test.csv'
test_label_file_path = path + '/data/test_labels.csv'

train_df, valid_df = data_loader.load_train_data(train_file_path, valid_rate=0.1, is_df=True)
test_df = data_loader.load_test_data(test_file_path, test_label_file_path, is_df=True)

In [0]:
classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [10]:
print("train_df.shape: ", train_df.shape)
print("valid_df.shape: ", valid_df.shape)
print("test_df.shape: ", test_df.shape) 

train_df.shape:  (143645, 8)
valid_df.shape:  (15926, 8)
test_df.shape:  (63978, 8)


## Preprocessing

### Text Cleaning
1. Lowercasing
2. Remove punctuation
3. Remove marginal white space or newline
      - strip left and right
      - \n
4. Remove stop words
5. Lemmatization

### After clean up, remove empty string

In [0]:
lemtzr = WordNetLemmatizer() 

def dataCleaning(row):
  txt = row['comment_text']
  txt = txt.lower() 
  
  ## expand contraction 
  ## code from https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
  txt = re.sub(r"won\'t", "will not", txt)
  txt = re.sub(r"can\'t", "can not", txt)
  txt = re.sub(r"n\'t", " not", txt)
  txt = re.sub(r"\'re", " are", txt)
  txt = re.sub(r"\'s", " is", txt)
  txt = re.sub(r"\'d", " would", txt)
  txt = re.sub(r"\'ll", " will", txt)
  txt = re.sub(r"\'t", " not", txt)
  txt = re.sub(r"\'ve", " have", txt)
  txt = re.sub(r"\'m", " am", txt)
  
  txt = re.sub('\W', ' ', txt)                                                  # remove non word characters
  txt = re.sub('\s+', ' ', txt)                                                 # remove white spaces
  
  txt = ' '.join([lemtzr.lemmatize(wrd) for wrd in txt.split(' ')])             # lemmatize
  return txt.strip()

In [0]:
train_df['filt_comment'] = train_df.apply(dataCleaning, axis = 1)
valid_df['filt_comment'] = valid_df.apply(dataCleaning, axis = 1)
test_df['filt_comment'] = test_df.apply(dataCleaning, axis = 1)

In [0]:
saveobj([train_df, valid_df, test_df], 'data/filtered_comment_pickle')